In [48]:
import contextlib
import enum
import json
import os
import pathlib
import typing as tp
import uuid

import numpy as np
import pandas as pd
import sklearn
from scipy.stats import mode
from sklearn.metrics._scorer import _check_multimetric_scoring
from sklearn.model_selection._validation import _score
from collections import Counter

In [49]:
def gini(y: np.ndarray) -> float:

    """
    Computes Gini index for given set of labels
    :param y: labels
    :return: Gini impurity
    """
    _, counts = np.unique(y, return_counts=True)
    p = counts / counts.sum()
    gini_index = 1 - np.sum(p**2)

#    raise NotImplementedError("COMPLETE THIS FUNCTION")
    return gini_index

In [50]:
y = np.array([0, 1, 2, 3, 4, 5])
res = gini(y)
res

np.float64(0.8333333333333333)

In [51]:
counter = Counter(y)
print(np.array(counter.values()))

dict_values([1, 1, 1, 1, 1, 1])


In [52]:
def weighted_impurity(y_left: np.ndarray, y_right: np.ndarray) -> \
        tp.Tuple[float, float, float]:
    """
    Computes weighted impurity by averaging children impurities
    :param y_left: left  partition
    :param y_right: right partition
    :return: averaged impurity, left child impurity, right child impurity
    """
    weighted_impurity = (y_left.shape[0] * gini(y_left) + y_right.shape[0] * gini(y_right)) / (y_left.shape[0] + y_right.shape[0])
    left_impurity = gini(y_left)
    right_impurity = gini(y_right)
#    raise NotImplementedError("COMPLETE THIS FUNCTION")
    return weighted_impurity, left_impurity, right_impurity

In [53]:
res = weighted_impurity(y[:3], y[3:])
res

(np.float64(0.6666666666666666),
 np.float64(0.6666666666666667),
 np.float64(0.6666666666666667))

In [54]:
def create_split(feature_values: np.ndarray, threshold: float) -> tp.Tuple[np.ndarray, np.ndarray]:
    """
    splits given 1-d array according to relation to threshold into two subarrays
    :param feature_values: feature values extracted from data
    :param threshold: value to compare with
    :return: two sets of indices
    """
    left_idx = [i for i in range(feature_values.shape[0]) if feature_values[i] <= threshold]
    right_idx = [i for i in range(feature_values.shape[0]) if feature_values[i] > threshold]
    #raise NotImplementedError("COMPLETE THIS FUNCTION")
    return left_idx, right_idx

In [55]:
l, r = create_split(y, 3)
print(y)
print(l)
print(r)

[0 1 2 3 4 5]
[0, 1, 2, 3]
[4, 5]


In [56]:

def _best_split(self, X: np.ndarray, y: np.ndarray):
    """
    finds best split
    :param X: Data, passed to node
    :param y: labels
    :return: best feature, best threshold, left child impurity, right child impurity
    """
    lowest_impurity = np.inf
    best_feature_id = None
    best_threshold = None
    lowest_left_child_impurity, lowest_right_child_impurity = None, None
    features = self._meta.rng.permutation(X.shape[1])
    for feature in features:
        current_feature_values = X[:, feature]
        thresholds = np.unique(current_feature_values)
        for threshold in thresholds:
            # find indices for split with current threshold
            left_inds, right_inds = create_split(current_feature_values, threshold)
            y_left = np.array([y[i] for i in left_inds])
            y_right = np.array([y[i] for i in right_inds])
            current_weighted_impurity, current_left_impurity, current_right_impurity = weighted_impurity(y_left, y_right)  # calcualte impurity for given indices
            #raise NotImplementedError("COMPLETE THIS FUNCTION")
            if current_weighted_impurity < lowest_impurity:
                lowest_impurity = current_weighted_impurity
                best_feature_id = feature
                best_threshold = threshold
                lowest_left_child_impurity = current_left_impurity
                lowest_right_child_impurity = current_right_impurity

    return best_feature_id, best_threshold, lowest_left_child_impurity, lowest_right_child_impurity

In [57]:
class NodeType(enum.Enum):
    REGULAR = 1
    TERMINAL = 2


In [58]:
class MyDecisionTreeNode:
    """
    Auxiliary class serving as representation of a decision tree node
    """

    def __init__(
            self,
            meta: 'MyDecisionTreeClassifier',
            depth,
            node_type: NodeType = NodeType.REGULAR,
            predicted_class: tp.Optional[tp.Union[int, str]] = None,
            left_subtree: tp.Optional['MyDecisionTreeNode'] = None,
            right_subtree: tp.Optional['MyDecisionTreeNode'] = None,
            feature_id: int = None,
            threshold: float = None,
            impurity: float = np.inf
    ):
        """

        :param meta: object, holding meta information about tree
        :param depth: depth of this node in a tree (is deduced on creation by depth of ancestor)
        :param node_type: 'regular' or 'terminal' depending on whether this node is a leaf node
        :param predicted_class: class label assigned to a terminal node
        :param feature_id: index if feature to split by
        :param
        """
        self._node_type = node_type
        self._meta = meta
        self._depth = depth
        self._predicted_class = predicted_class
        self._class_proba = None
        self._left_subtree = left_subtree
        self._right_subtree = right_subtree
        self._feature_id = feature_id
        self._threshold = threshold
        self._impurity = impurity

    def _best_split(self, X: np.ndarray, y: np.ndarray):
        """
        finds best split
        :param X: Data, passed to node
        :param y: labels
        :return: best feature, best threshold, left child impurity, right child impurity
        """
        lowest_impurity = np.inf
        best_feature_id = None
        best_threshold = None
        lowest_left_child_impurity, lowest_right_child_impurity = None, None
        features = self._meta.rng.permutation(X.shape[1])
        for feature in features:
            current_feature_values = X[:, feature]
            thresholds = np.unique(current_feature_values)
            for threshold in thresholds:
                # find indices for split with current threshold
                left_inds, right_inds = create_split(current_feature_values, threshold)
                y_left = np.array([y[i] for i in left_inds])
                y_right = np.array([y[i] for i in right_inds])
                current_weighted_impurity, current_left_impurity, current_right_impurity = weighted_impurity(y_left, y_right)  # calcualte impurity for given indices
                #raise NotImplementedError("COMPLETE THIS FUNCTION")
                if current_weighted_impurity < lowest_impurity:
                    lowest_impurity = current_weighted_impurity
                    best_feature_id = feature
                    best_threshold = threshold
                    lowest_left_child_impurity = current_left_impurity
                    lowest_right_child_impurity = current_right_impurity

        return best_feature_id, best_threshold, lowest_left_child_impurity, lowest_right_child_impurity

    def fit(self, X: np.ndarray, y: np.ndarray, all_classes = None):
        """
        recursively fits a node, providing it with predicted class or split condition
        :param X: Data
        :param y: labels
        :return: fitted node
        """

        if all_classes == None:
            all_classes = np.unique(y)
        #raise NotImplementedError("COMPLETE THIS FUNCTION")
        if (
                len(set(y)) == 1 
                or self._meta.min_samples_split > X.shape[0] 
                or (self._meta.max_depth is not None and self._depth >= self._meta.max_depth)
                 # complete all conditions by which this node should be terminal
        ):
            self._node_type = NodeType.TERMINAL
            general_counts = {c:0 for c in all_classes}
            values,counts = np.unique(y, return_counts=True)      
            for c, count in zip(values, counts):
                general_counts[c] = count / len(y)
            self._predicted_class = values[counts.argmax()] # choose most common class
    #        proba = [0.0 for i in range(self._meta._n_classes)]
    #        for c, count in zip([i for i in range(len(y))],counts):
   #             proba[c] = count / len(y)
            self._class_proba = list(general_counts.values()) # vector of probabilities of all classes with index from 0 to n_classes-1 (_n_classes from tree class)
            return self

        self._feature_id, self._threshold, left_imp, right_imp = self._best_split(X, y)
        left_idx, right_idx = create_split(X[:, self._feature_id], self._threshold)
        self._left_subtree = MyDecisionTreeNode(
            meta=self._meta,
            depth=self._depth+1,  # adjust depth
            impurity=left_imp
        ).fit(
            np.array([X[i] for i in left_idx]),
            np.array([y[i] for i in left_idx]), # choose proper data to fit
            all_classes=all_classes
        )
        self._right_subtree = MyDecisionTreeNode(
            meta=self._meta,
            depth=self._depth+1,  # adjust depth
            impurity=right_imp
        ).fit(
            np.array([X[i] for i in right_idx]),
            np.array([y[i] for i in right_idx]),
            all_classes=all_classes
             #  choose data to fit
        )
        return self

    def predict(self, x: np.ndarray):
        """
        Predicts class for a single object
        :param x: object of shape (n_features, )
        :return: class assigned to object
        """
 #       raise NotImplementedError("COMPLETE THIS FUNCTION")
        if self._node_type is NodeType.TERMINAL:
            return self._predicted_class
        if x[self._feature_id] <= self._threshold:
            return self._left_subtree.predict(x)
            #pass  # look for an answer recursively
        else:
            return self._right_subtree.predict(x)

    def predict_proba(self, x: np.ndarray):
        """
        Predicts probability for a single object
        :param x: object of shape (n_features, )
        :return: vector of probabilities assigned to object
        """
#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        if self._node_type is NodeType.TERMINAL:
            return self._class_proba
        if x[self._feature_id] <= self._threshold:
            return self._left_subtree._class_proba  # look for an answer recursively
        else:
            return self._right_subtree._class_proba


In [59]:
class MyDecisionTreeClassifier:
    """
    Class analogous to sklearn implementation of decision tree classifier with Gini impurity criterion,
    named in a manner avoiding collisions
    """

    def __init__(
            self,
            max_depth: tp.Optional[int] = None,
            min_samples_split: tp.Optional[int] = 2,
            seed: int = 0
    ):
        """
        :param max_depth: maximal depth of tree, prevents overfitting
        :param min_samples_split: minimal amount of samples for node to be a splitter node
        :param seed: seed for RNG, enables reproducibility
        """
        self.root = MyDecisionTreeNode(self, 1)
        self._is_trained = False
        self.max_depth = max_depth or np.inf
        self.min_samples_split = min_samples_split or 2
        self.rng = np.random.default_rng(seed)
        self._n_classes = 0

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        starts recursive process of node criterion fitting from the root
        :param X: Data
        :param y: labels
        :return: fitted self
        """
        self._n_classes = np.unique(y).shape[0]
        self.root.fit(X, y)
        self._is_trained = True
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predicts class for a sequence of objects
        :param x: Data
        :return: classes assigned to each object
        """
        if not self._is_trained:
            raise RuntimeError('predict call on untrained model')
        else:
            #raise NotImplementedError("COMPLETE THIS FUNCTION")
            # predict class for each object
            return np.array([self.root.predict(s) for s in X])

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Predicts class for a sequence of objects
        :param x: Data
        :return: probabilities of all classes for each object
        """
        if not self._is_trained:
            raise RuntimeError('predict call on untrained model')
        else:
            res = np.array([])
            for s in X:
                res = np.append(res, self.root.predict_proba(s))
            return res
#            raise NotImplementedError("COMPLETE THIS FUNCTION")
            # predict class probabilities for each object


In [60]:
tree = MyDecisionTreeClassifier()
tree.fit(np.empty((1,2)), np.array([0]))

In [61]:
p = tree.predict_proba(np.empty((1,2)))
print(p)

[1.]


In [64]:
class MyRandomForestClassifier:
    """
    Data-diverse ensemble of tree calssifiers
    """
    big_number = 1 << 32

    def __init__(
            self,
            n_estimators: int,
            max_depth: tp.Optional[int] = None,
            min_samples_split: tp.Optional[int] = 2,
            seed: int = 0
    ):
        """
        :param n_estimators: number of trees in forest
        :param max_depth: maximal depth of tree, prevents overfitting
        :param min_samples_split: minimal amount of samples for node to be a splitter node
        :param seed: seed for RNG, enables reproducibility
        """
        self._n_classes = 0
        self._is_trained = False
        self.rng = np.random.default_rng(seed)
        self.estimators = [
            MyDecisionTreeClassifier(max_depth, min_samples_split, seed=seed) for
            seed in self.rng.choice(max(MyRandomForestClassifier.big_number, n_estimators), size=(n_estimators,),
                                    replace=False)]

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray):
        """
        returns bootstrapped sample from X of equal size
        :param X: objects collection to sample from
        :param y: corresponding labels
        :return:
        """

#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        indices = self.rng.integers(0, len(X), size=len(X))  # bootstrap indices with equal probability from X
        return X[indices], y[indices]

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        fits each estimator of the ensemble on the bootstrapped data sample
        :param X: Data
        :param y: labels
        :return: fitted self
        """
        self._n_classes = np.unique(y).shape[0]
#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        for estimator in self.estimators:
            bootstarped_x, bootstraped_y = self._bootstrap_sample(X, y) # fit each estimator on a bootstrapped sample
            estimator.fit(bootstarped_x, bootstraped_y)
        self._is_trained = True
        return self

    def predict_proba(self, X: np.ndarray):
        """
        predict probability of each class by averaging over all base estimators
        :param X: Data
        :return: array of probabilities
        """
        probas = np.zeros((X.shape[0], self._n_classes))
        all_probas = np.array([tree.predict_proba(X) for tree in self.estimators])

        probas = np.mean(all_probas, axis=0)
#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        # get averaged probabilities from each estimator
        return probas

    def predict(self, X):
        """
        predict class for each object
        :param X: Data
        :return: array of class labels
        """
#        raise NotImplementedError("COMPLETE THIS FUNCTION")

        return np.argmax(self.predict_proba(X), axis=1) # most probable class for each object


In [65]:
class ExperimentHandler:
    """This class perfoms experiments with given model, measures metrics and logs everything for thorough comparison"""
    stacking_prediction_filename = 'cv_stacking_prediction.npy'
    test_stacking_prediction_filename = 'test_stacking_prediction.npy'

    def __init__(
            self,
            X_train: pd.DataFrame,
            y_train: pd.Series,
            X_test: pd.DataFrame,
            y_test: pd.Series,
            cv_iterable: tp.Union[sklearn.model_selection.KFold, tp.Iterable],
            logger: Logger,
            metrics: tp.Dict[str, tp.Union[tp.Callable, str]],
            n_jobs=-1
    ):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self._cv_iterable = cv_iterable
        self.logger = logger
        self._metrics = metrics
        self._n_jobs = n_jobs

    def score_test(self, estimator, metrics, run, test_data=None):
        """
        Computes scores for test data and logs them to given run
        :param estimator: fitted estimator
        :param metrics: metrics to compute
        :param run: run to log into
        :param test_data: optional argument if one wants to pass augmented test dataset
        :return: None
        """
        if test_data is None:
            test_data = self.X_test
        test_scores = _score(estimator, test_data, self.y_test, metrics)
        run.log_values({key + '_test': value for key, value in test_scores.items()})

    def score_cv(self, estimator, metrics, run):
        """
        computes scores on cross-validation
        :param estimator: estimator to fit
        :param metrics: metrics to compute
        :param run: run to log to
        :return: None
        """
#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        cross_val_results = sklearn.model_selection.cross_val_score(estimator=estimator, X=self.X_train, y=self.y_train, cv=self._cv_iterable, scoring=metrics)  # compute crossval scores (easier done using corresponding sklearn method)
        for key, value in cross_val_results.items():
            if key.startswith('test_'):
                metric_name = key.split('_', maxsplit=1)[1]
                mean_score = np.mean(value)
                std_score = np.std(value)
                run.log_values(
                    {
                        metric_name + '_mean': mean_score,
                        metric_name + '_std': std_score
                    }
                )

    def generate_stacking_predictions(self, estimator, run):
        """
        generates predictions over cross-validation folds, then saves them as artifacts
        returns fitted estimator for convinience and les train overhead
        :param estimator: estimator to use
        :param run: run to log to
        :return: estimator fitted on train, stacking cross-val predictions, stacking test predictions
        """
#        raise NotImplementedError("COMPLETE THIS FUNCTION")
        if hasattr(estimator, "predict_proba"):  # choose the most informative method for stacking predictions
            method = "predict_proba"
        elif hasattr(estimator, "decision_function"):
            method = "decision_function"
        else:
            method = "predict"
        cross_val_stacking_prediction = sklearn.model_selection.cross_val_predict(estimator=estimator, X=self.X_test, y=self.y_test, cv=self._cv_iterable, method=method)  # generate crossval predictions for stacking using most informative method
        run.log_artifact(ExperimentHandler.stacking_prediction_filename,
                         lambda file: np.save(file, cross_val_stacking_prediction))
        estimator.fit(self.X_train, self.y_train)
        test_stacking_prediction = getattr(estimator, method)(self.X_test)
        run.log_artifact(ExperimentHandler.test_stacking_prediction_filename,
                         lambda file: np.save(file, test_stacking_prediction))
        return estimator, cross_val_stacking_prediction, test_stacking_prediction

    def get_metrics(self, estimator):
        """
        get callable metrics with estimator validation
        (e.g. estimator has predict_proba necessary for likelihood computation, etc)
        """
        return _check_multimetric_scoring(estimator, self._metrics)

    def run(self, estimator: sklearn.base.BaseEstimator, name=None):
        """
        perform run for given estimator
        :param estimator: estimator to use
        :param name: name of run for convinience and consitent logging
        :return: leaderboard with conducted run
        """
        metrics = self.get_metrics(estimator)
        with self.logger.run(name=name) as run:
            # compute predictions over cross-validation
            self.score_cv(estimator, metrics, run)
            fitted_on_train, _, _ = self.generate_stacking_predictions(estimator, run)
            self.score_test(fitted_on_train, metrics, run, test_data=self.X_test)
            return self.logger.leaderboard.loc[[run.name]]

    def get_stacking_predictions(self, run_names):
        """
        :param run_names: run names for which to extract stacking predictions for averaging and stacking
        :return: dataframe with predictions indexed by run names
        """
        train_dataframes = []
        test_dataframes = []
        for run_name in run_names:
            train_filename = self.logger.path / run_name / ExperimentHandler.stacking_prediction_filename
            train_dataframes.append(load_predictions_dataframe(filename=train_filename, column_prefix=run_name,
                                                               index=self.X_train.index))
            test_filename = self.logger.path / run_name / ExperimentHandler.test_stacking_prediction_filename
            test_dataframes.append(load_predictions_dataframe(filename=test_filename, column_prefix=run_name,
                                                              index=self.X_test.index))

        return pd.concat(train_dataframes, axis=1), pd.concat(test_dataframes, axis=1)


In [ ]:
handler = ExperimentHandler()

TypeError: ExperimentHandler.__init__() missing 7 required positional arguments: 'X_train', 'y_train', 'X_test', 'y_test', 'cv_iterable', 'logger', and 'metrics'